In [253]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By  
import time
import urllib

In [220]:
def download_slp(url_loc, dl = True):
    with urllib.request.urlopen(url_loc) as url:
        name = url_loc[url_loc.find('_') + 1:]
        if dl == True:
            data = open("Data/" + name, 'wb')
            data.write(url.read())
            data.close()
        else:
            print(name)

In [221]:
def page_download(soup):
    empty = True
    for i, link in enumerate(soup.findAll('a')):
        file_url = link.get('href')
        if not (file_url is None) and (file_url[:30] == 'https://storage.googleapis.com'):
            download_slp(file_url.replace(' ','%20'), False)
            if empty:
                empty = False
    return empty

In [216]:
def scrape_page(base_url, num, driver):    
    url = base_url + '?page=' + str(num)
    driver.get(url)
    print(url)
    html = driver.page_source
    soup = bs(html)
    page_empty = page_download(soup)
    return page_empty

In [217]:
def scrape_tour(base_url, tour_code):
    driver = webdriver.Firefox()
    
    url = base_url + tour_code
    driver.get(url)
    page_empty = False
    i = 2
    while not page_empty:
        print(i)
        page_empty = scrape_page(url, i, driver)
        i += 1

In [219]:
_URL = 'https://slippi.gg/tournament/' 
scrape_tour(_URL,'0x7')

2
https://slippi.gg/tournament/0x7?page=2


In [263]:
driver = webdriver.Firefox()
url = 'https://slippi.gg/' 
driver.get(url)
#
button_ui = driver.find_element_by_xpath('/html/body/div[1]/div/header/div/header/div/button')
button_ui.click()
#tournament button
button_tournament = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div/ul/div[2]')
button_tournament.click()

time.sleep(5)

tournament_buttons = driver.find_elements_by_link_text("Browse all games")
tournament_buttons
# driver.find_element_by_link_text("Browse all games").click()
# button = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div/div/div[1]/div/div[2]/div[3]/a')
# buttons = driver.find_element_by_xpath('//a[contains(@href,"href")]')
# buttons

#for tournament page
    #get list of buttons
    #for button
        #enter page
        #scrape page
        #next page
        #return

C:\Users\steph\AppData\Local\Temp/ipykernel_18820/319890485.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button_ui = driver.find_element_by_xpath('/html/body/div[1]/div/header/div/header/div/button')
C:\Users\steph\AppData\Local\Temp/ipykernel_18820/319890485.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button_tournament = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div/ul/div[2]')
C:\Users\steph\AppData\Local\Temp/ipykernel_18820/319890485.py:13: DeprecationWarning: find_elements_by_link_text is deprecated. Please use find_elements(by=By.LINK_TEXT, value=text) instead
  buttons = driver.find_elements_by_link_text("Browse all games")


[<selenium.webdriver.remote.webelement.WebElement (session="5e01ed3b-c4e1-43cf-a3c2-77515bc87656", element="b34bf26f-9f3a-42e3-b531-574018dc5700")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5e01ed3b-c4e1-43cf-a3c2-77515bc87656", element="4b11428c-eabe-4712-adf3-b5750096b2d0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5e01ed3b-c4e1-43cf-a3c2-77515bc87656", element="7369043e-f900-440d-a4f8-fbbd8d1a3018")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5e01ed3b-c4e1-43cf-a3c2-77515bc87656", element="b5cc317a-5447-4bd5-8832-56e7ea929f2d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5e01ed3b-c4e1-43cf-a3c2-77515bc87656", element="8e45ba37-b75d-4f99-9847-4c2b90503773")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5e01ed3b-c4e1-43cf-a3c2-77515bc87656", element="87e152e9-06a1-4330-84ad-d3bc36258612")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5e01ed3b-c4e1-43cf-a3c2-77515bc87656", element